<a href="https://colab.research.google.com/github/eliazarnd/GreenPhoto/blob/master/Entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import sys
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K

from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [151]:

#Reiniciando la sesion
K.clear_session()

data_entrenamiento = '/content/gdrive/My Drive/Machine Learning/Clasificador/data/flowers/data/entrenamiento'
data_validacion = '/content/gdrive/My Drive/Machine Learning/Clasificador/data/flowers/data/validacion'

dir = '/content/gdrive/My Drive/Machine Learning/Clasificador/data/flowers/data'

d = os.path.exists(dir)
print(d)

True


In [0]:
#Iniciando las variables de entrada
#Parametros
epocas = 2
altura, longitud = 100, 100
batch_size = 32
pasos = 1000
pasos_validacion = 200
filtros_Conv1 = 32
filtros_Conv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 5
lr = 0.00005

In [153]:
#Preprocesamiento de la imagenes de las flores
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1. /255,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True
)

validacion_datagen = ImageDataGenerator(
    rescale = 1. /255
)

imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)
imagen_validacion = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)



Found 2288 images belonging to 5 classes.
Found 2035 images belonging to 5 classes.


In [156]:
#Crear la red CNN
cnn = Sequential()

#Se agrega el primer filtro de convolucion
cnn.add(Convolution2D(
    filtros_Conv1, 
    tamano_filtro1, 
    padding= 'same', 
    input_shape = (altura, longitud, 3), 
    activation = 'tanh'))

#Se reliza el max Polling para reducir las imagenes
cnn.add(MaxPooling2D(pool_size = tamano_pool))

#Se vuelve a realizar otro filtro de convolucion
cnn.add(Convolution2D(
    filtros_Conv2,
    tamano_filtro2,
    padding = 'same',
    activation = 'tanh'
))
#Se realiza otro max Pooling para reducir la imagen
cnn.add(MaxPooling2D(pool_size = tamano_pool))

#Se mete toda esa informacion en un solo vector
cnn.add(Flatten())

#Se crean 256 neuronas las cuales se activan con la funcion Relu
cnn.add(Dense(256, activation = 'tanh'))

#Desecha la mitad de las imagenes para que se sobreajuste la red
cnn.add(Dropout(0.5))

cnn.add(Dense(clases, activation = 'softmax'))

cnn.compile(loss = 'categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics = ['accuracy'])


cnn.fit(imagen_entrenamiento, 
        steps_per_epoch = pasos, 
        epochs = epocas, 
        validation_data = imagen_validacion,
        validation_steps = pasos_validacion
       )

path = '/content/gdrive/My Drive/Machine Learning/Clasificador/data/flowers/data'

cnn.save('content/gdrive/My Drive/Machine Learning/Clasificador/modelo/modelo_Fl.h5')
cnn.save_weights('content/gdrive/My Drive/Machine Learning/Clasificador/modelo/pesos_Fl.h5')


Epoch 1/2
72/72 [==============================] - 21s 297ms/step - loss: 1.4778 - acc: 0.3816 - val_loss: 1.3513 - val_acc: 0.4339
Epoch 2/2
72/72 [==============================] - 20s 274ms/step - loss: 1.2607 - acc: 0.4781 - val_loss: 1.4410 - val_acc: 0.4319


OSError: ignored